In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

### Récupérer les noms de 50 Etats aux USA sur wikipedia

In [2]:
# L'URL à scraper est la page Wikipedia qui présente les 50 Etats américains
url_states_all = 'https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States'

In [3]:
# Une requête est effectuée vers le serveur afin d'établir une connexion avec ce dernier
res = requests.get(url_states_all)
print(res)

<Response [200]>


In [4]:
# Un objet "soup" va contenir l'ensemble du code source de la page Wikipédia
soup = BeautifulSoup(res.content, 'html.parser')
print(soup)

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of states and territories of the United States - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"e69fc0d9-1103-4666-b5ad-261a6d08f5a6","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_states_and_territories_of_the_United_States","wgTitle":"List of states and territories of the United States","wgCurRevisionId":975758572,"wgRevisionId":975758572,"wgArticleId":12610470,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: unfit url","CS1 Spanish-language sources (

In [5]:
# Dans le code source, on récupère la balise "table" qui contient le tableau avec les informations utiles
table = soup.find('table', attrs={'class','wikitable sortable plainrowheaders'})
table

<table class="wikitable sortable plainrowheaders" style="text-align: center;">
<caption>States of the United States of America
</caption>
<tbody><tr>
<th colspan="2" rowspan="2" scope="col">Flag, name and<br/><a class="mw-redirect" href="/wiki/List_of_U.S._state_abbreviations" title="List of U.S. state abbreviations">postal abbreviation</a><sup class="reference" id="cite_ref-USPSabbreviations_14-0"><a href="#cite_note-USPSabbreviations-14">[12]</a></sup>
</th>
<th colspan="2" scope="col">Cities
</th>
<th rowspan="2" scope="col">Ratification or<br/>admission<sup class="reference" id="cite_ref-16"><a href="#cite_note-16">[C]</a></sup>
</th>
<th rowspan="2" scope="col">Population<br/><sup class="reference" id="cite_ref-17"><a href="#cite_note-17">[D]</a></sup><sup class="reference" id="cite_ref-AnnualEstUS_18-0"><a href="#cite_note-AnnualEstUS-18">[14]</a></sup>
</th>
<th colspan="2" scope="col">Total area<sup class="reference" id="cite_ref-areameasurements_19-0"><a href="#cite_note-aream

Pour obtenir le nom de l'Etat, il faut suivre le chemin suivant :

- Balise "table" : l'élément de tableau
- Balise "tbody" : élément de corps d'un tableau qui regroupe plusieurs "tr"
- Balises "tr" : une ligne de cellules dans un tableau
- Balises "th" : cellule d'en-tête pour un groupe de cellule
- Balise "a" : élément d'ancre qui renvoie vers la page Wikipédia de l'Etat, mais dont le titre nous intéresse (cf. nom de l'Etat)


Donc la boucle suivante va parcourir chaque balise "tr", puis la balise "th" puis la balise "a" qui contient le nom de l'Etat. Toute cela sera ajouté dans une liste "data".

In [14]:
data = []

for th in table.find('tbody').find_all('tr'):
    a = th.find('th').find('a')
    if a != None:
        data.append(a.get_text())

In [15]:
# On supprime le premier élément qui n'est pas le nom d'un état
data = data[1:]
data

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

### Récupérer les nom des capitales des 50 Etats ainsi que les coordonnées GPS

#### Trouver le nom de la capitale de l'Etat de Californie ainsi que ses coordonnées GPS


Initialisons d'abord un driver Selenium. Pour cela, nous avons besoin de la version executable du navigateur internet (Chrome ici, sous le nom de "chromedriver.exe"). Elle sera placée dans le même dossier que le notebook afin qu'elle puisse être facilement trouvable.

In [17]:
from pathlib import *
import time

chrome_path = Path.cwd() / 'chromedriver.exe'
url_alabama = 'https://en.wikipedia.org/wiki/California'

# On se rend sur l'URL à scraper en initialisant le "driver", puis en se rendant sur l'URL avec la méthode "get(URL)"
driver = webdriver.Chrome(executable_path=chrome_path.as_posix())
driver.get(url_alabama)

# On crée une variable avec le XPATH du lien qui renvoie vers la capitale de l'Etat. 
capital_path = '/html/body/div[3]/div[3]/div[4]/div/table[1]/tbody/tr[12]/td/a'

# Puis on demande à Selenium trouver ce lien sur la page grâce à la méthode "find_element_by_xpath(xpath)"
capital = driver.find_element_by_xpath(capital_path)

# Puis on demande à Selenium de cliquer sur le lien qu'il a trouvé sur la page après avoir printé le nom de la capitale
print(capital.text)
capital.click()

time.sleep(1/2)

# On récupère le selecteur CSS du lien qui renvoie vers le site GeoHack
coord_path = "span.plainlinks:nth-child(2) > span:nth-child(1) > a:nth-child(2) > span:nth-child(1)"

# Puis on demande à selenium de repérer le lien sur la page puis de cliquer dessus
coord = driver.find_element_by_css_selector(coord_path)
coord.click()

# Sur la page, on récupère le XPATH de la latitude et le XPATH de la longitude 
lat_path = '/html/body/div[2]/div[1]/div/div[1]/div/div[1]/table/tbody/tr[1]/td[1]/span[3]/span[1]'
lon_path = '/html/body/div[2]/div[1]/div/div[1]/div/div[1]/table/tbody/tr[1]/td[1]/span[3]/span[2]'

# Puis on demande à Selenium de les récupérer et de les printer tout en les transformant en float
lat = driver.find_element_by_xpath(lat_path)
lon = driver.find_element_by_xpath(lon_path)

print(float(lat.text), float(lon.text))

driver.quit()

Sacramento
38.581667 -121.494444


#### Boucle pour trouver la capitale des 50 Etats ainsi que leur coordonnées GPS

In [18]:
# Initialisation du Diver
driver = webdriver.Chrome(executable_path=chrome_path.as_posix())
url_state = 'https://en.wikipedia.org/wiki/{}'

# Reprise des XPATH et selecteur CSS définis plus haut 
capital_path = '/html/body/div[3]/div[3]/div[4]/div/table[1]/tbody/tr[12]/td/a'
coord_path = "span.plainlinks:nth-child(2) > span:nth-child(1) > a:nth-child(2) > span:nth-child(1)"
lat_path = '/html/body/div[2]/div[1]/div/div[1]/div/div[1]/table/tbody/tr[1]/td[1]/span[3]/span[1]'
lon_path = '/html/body/div[2]/div[1]/div/div[1]/div/div[1]/table/tbody/tr[1]/td[1]/span[3]/span[2]'

# Listes vides qui recevront le nom de la capitale, les coordonées GPS et l'Etat sur lequel la boucle a itéré
capital_list = []
coord_list = []
state_check = []

for i in data:
# La recherche de l'URL se fait grâce à la méthode format qui remplace le dernier terme de l'URL par le nom de l'Etat 
# On ajoute également, dans le nom de l'URL, la chaîne de caractère '_(U.S._state)' afin d'éviter les erreurs dans Wikipedia 

    driver.get(url_state.format(i + '_(U.S._state)'))
    capital = driver.find_element_by_xpath(capital_path)
    capital_list.append(capital.text)
    
    capital.click()
        
    time.sleep(1/2)
    
    try:
        coord = driver.find_element_by_css_selector(coord_path)
    
    except:
        coord_tupple = (None,None)
        coord_list.append(coord_tupple)
        state_check.append(i)
        continue
    
    try:
        coord.click()
    
    except:
        coord_tupple = (None,None)
        coord_list.append(coord_tupple)
        state_check.append(i)
        continue
    
    time.sleep(1/2)

    lat = driver.find_element_by_xpath(lat_path)
    lon = driver.find_element_by_xpath(lon_path)
        
    coord_tupple = (float(lat.text),float(lon.text))
    coord_list.append(coord_tupple)
        
    state_check.append(i)

driver.quit()

In [19]:
# Liste des capitales
capital_list

['Montgomery',
 'Juneau',
 'Phoenix',
 'Little Rock',
 'Sacramento',
 'Denver-Aurora-Lakewood, CO MSA',
 'Hartford',
 'Dover',
 'Tallahassee',
 'Atlanta',
 'Honolulu',
 'Boise',
 'Springfield',
 'Indianapolis',
 'Des Moines',
 'Topeka',
 'Frankfort',
 'Baton Rouge',
 'Augusta',
 'Annapolis',
 'Boston',
 'Lansing',
 'Saint Paul',
 'Jackson',
 'Jefferson City',
 'Helena',
 'Lincoln',
 'Carson City',
 'Concord',
 'Newark',
 'Santa Fe',
 'Albany',
 'Raleigh',
 'Bismarck',
 'Greater Cincinnati',
 'Oklahoma City',
 'Salem',
 'Harrisburg',
 'Providence',
 'Columbia',
 'Pierre',
 'Nashville',
 'Austin',
 'Salt Lake City',
 'Montpelier',
 'Richmond',
 'Olympia',
 'Charleston',
 'Madison',
 'Cheyenne']

In [20]:
# Liste des états
state_check

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [21]:
# Liste des coordonnées
coord_list

[(32.361667, -86.279167),
 (None, None),
 (34.0, -112.0),
 (None, None),
 (38.581667, -121.494444),
 (None, None),
 (41.7625, -72.674167),
 (None, None),
 (30.455, -84.253333),
 (33.755, -84.39),
 (21.306944, -157.858333),
 (43.615833, -116.201667),
 (39.799444, -89.655),
 (40.0, -86.0),
 (42.0, -93.0),
 (39.055833, -95.689444),
 (38.2, -84.866667),
 (30.4475, -91.178611),
 (None, None),
 (38.973056, -76.501111),
 (42.358056, -71.063611),
 (42.733611, -84.546667),
 (46.0, -94.0),
 (33.0, -90.0),
 (None, None),
 (46.591111, -112.020278),
 (40.808889, -96.678889),
 (39.164444, -119.766944),
 (43.206667, -71.538056),
 (None, None),
 (35.667222, -105.964444),
 (42.6525, -73.757222),
 (35.766667, -78.633333),
 (46.808333, -100.783611),
 (39.0, -84.5),
 (35.468611, -97.521389),
 (None, None),
 (40.269722, -76.875556),
 (41.823611, -71.422222),
 (34.000556, -81.034722),
 (None, None),
 (36.166667, -86.783333),
 (30.267222, -97.743056),
 (39.0, -111.0),
 (None, None),
 (37.533333, -77.466667),

#### Création d'un dataset avec les informations récupérées

In [22]:
import pandas as pd

dataset = pd.DataFrame({
    'state' : state_check,
    'capital' : capital_list,
    'coord' : coord_list
})

dataset

,state,capital,coord
0,Alabama,Montgomery,"(32.361667, -86.279167)"
1,Alaska,Juneau,"(None, None)"
2,Arizona,Phoenix,"(34.0, -112.0)"
3,Arkansas,Little Rock,"(None, None)"
4,California,Sacramento,"(38.581667, -121.494444)"
5,Colorado,"Denver-Aurora-Lakewood, CO MSA","(None, None)"
6,Connecticut,Hartford,"(41.7625, -72.674167)"
7,Delaware,Dover,"(None, None)"
8,Florida,Tallahassee,"(30.455, -84.253333)"
9,Georgia,Atlanta,"(33.755, -84.39)"


#### Affichage des capitales sur une carte grâce aux coordonnées GPS obtenues


In [23]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [25]:
import folium

def generateBaseMap(default_location=[48, -102], default_zoom_start=4.5):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map_usa = generateBaseMap()

for coord in coord_list:
    try:
        folium.CircleMarker(location=([coord[0], coord[1]]), radius=10,color='darkblue',fill_color='darkblue').add_to(base_map_usa)
    except:
        continue

base_map_usa